In [1]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Reading Data

In [35]:
remove_chars = [chr(x) for x in range(31)]
def segmentWords(s): 
    for char in remove_chars:
        s = s.replace(char,"")
    return s.split()

def readFile(fileName):
    # Function for reading file
    # input: filename as string
    # output: contents of file as list containing single words
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = segmentWords('\n'.join(contents))
    return result

#### Create a Dataframe containing the counts of each word in a file

In [36]:
d = []

for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        words = readFile(directory + "/" + file)
        e = {x:words.count(x) for x in words}
        e['__FileID__'] = file
        e['is_positive'] = 1 if c == 'pos' else 0
        d.append(e)

Create a dataframe from d - make sure to fill all the nan values with zeros.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html


In [37]:
df = pd.DataFrame(data=d,index=None)
df.fillna(value=0,axis=0,inplace=True)
df = df.astype(np.int32, errors='ignore') # Become slightly more memory efficient, because my computer is a potato.
del d

In [38]:
df.head()

,!,"""",#,#05425,#1,#13,#2,#3,#5,#6,...,zukovsky,zundel,zurg's,zweibel,zwick,zwick's,zwigoff's,zycie,zycie',|
0,3,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

#### Split data into training and validation set 

* Sample 80% of your dataframe to be the training data

* Let the remaining 20% be the validation data (you can filter out the indicies of the original dataframe that weren't selected for the training data)

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [6]:
traindf, validatedf = train_test_split(df, test_size=0.2)

* Split the dataframe for both training and validation data into x and y dataframes - where y contains the labels and x contains the words

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [7]:
training_y = pd.DataFrame(traindf['is_positive']).values.ravel()
training_x = pd.DataFrame(traindf.drop('is_positive', axis = 1).drop('__FileID__', axis = 1))

valid_y = pd.DataFrame(validatedf['is_positive']).T.values.ravel()
valid_x = pd.DataFrame(validatedf.drop('is_positive', axis = 1).drop('__FileID__', axis = 1))


In [8]:
training_x.shape

(1280, 45641)

In [20]:
training_x["zundel"].sum()

2

NameError: name 'values' is not defined

# Logistic Regression

#### Basic Logistic Regression
* Use sklearn's linear_model.LogisticRegression() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [9]:
model = LogisticRegression()
trained_model = model.fit(training_x,training_y)


In [10]:
# make predictions
# summarize the fit of the model
accuracy_LR =  trained_model.score(valid_x, valid_y)
print(accuracy_LR)

0.840625


In [13]:
c_space = np.logspace(-3, 1, num=10, endpoint=True, base=10.0,)
param_grid = {'C': c_space}
logReg_model = LogisticRegression()
logReg_CV = GridSearchCV(estimator = logReg_model, param_grid = param_grid, cv = 5, n_jobs = -1, refit = True, verbose = 3)
trained_models = logReg_CV.fit(training_x, training_y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  5.2min finished


In [15]:
print("Tuned Logistic Regression Parameters: {}".format(trained_models.best_params_)) 
print("Best score is {}".format(trained_models.best_score_ ))

Tuned Logistic Regression Parameters: {'C': 1.5264179671752318}
Best score is 0.82109375


In [2]:
from sklearn.feature_selection import RFE
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)

#### Changing Parameters

In [46]:
model2 = LogisticRegression(C = 1.5264179671752318)
trained_model2 = model2.fit(training_x,training_y)


In [47]:
accuracy_LR2 =  trained_model2.score(valid_x, valid_y)
print(accuracy_LR2)

0.8375


#### Feature Selection
* In the backward stepsize selection method, you can remove coefficients and the corresponding x columns, where the coefficient is more than a particular amount away from the mean - you can choose how far from the mean is reasonable.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html

In [17]:
from sklearn.feature_selection import VarianceThreshold

# normalize the dataset


In [ ]:
from sklearn.feature_selection import RFECV

model3 = LogisticRegression()
rfecv = RFECV(estimator=model3, step=1, cv=StratifiedKFold(2),
              scoring='accuracy', n_jobs = -1, verbose = 1)
rfecv.fit(training_x, training_y)

How did you select which features to remove? Why did that reduce overfitting?

# Single Decision Tree

#### Basic Decision Tree

* Initialize your model as a decision tree with sklearn.
* Fit the data and labels to the model.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


#### Changing Parameters
* To test out which value is optimal for a particular parameter, you can either loop through various values or look into sklearn.model_selection.GridSearchCV

References:


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

How did you choose which parameters to change and what value to give to them? Feel free to show a plot.

Why is a single decision tree so prone to overfitting?

# Random Forest Classifier

#### Basic Random Forest

* Use sklearn's ensemble.RandomForestClassifier() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


#### Changing Parameters

What parameters did you choose to change and why?

How does a random forest classifier prevent overfitting better than a single decision tree?